In [ ]:
import torch

x = torch.tensor([1.0, 2.0, 3.0, 4.0])
x.requires_grad_()

input = torch.randn((16,5,4,4))
x = torch.tril(torch.ones(4, 4))
input.masked_fill_(x == 0, float('-inf'))
input

In [27]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TRITON_INTERPRET'] ='1'
import torch
import triton
import triton.language as tl

@triton.jit
def add_one(input, output,stride_bs,stride_h,stride_seq,stride_d):
    block_id = tl.program_id(0)
    bs = 12
    h = 3
    offset_bsh = bs*stride_bs+h*stride_h
    block_x = tl.make_block_ptr(
        base= input+offset_bsh,
        shape = (8,4),
        strides=(stride_seq,stride_d),
        offsets=(block_id*2,0),#block_id * block_size
        block_shape=(2,4),
        order=(1,0),
    )
    block_y = tl.make_block_ptr(
        base= output+offset_bsh,
        shape = (8,4),
        strides=(stride_seq,stride_d),
        offsets=(block_id*2,0),#block_id * block_size
        block_shape=(2,4),
        order=(1,0),
    )

    x = tl.load(block_x)
    x+=1
    tl.store(block_y, x,boundary_check=(0,1))


x = torch.arange(32,dtype= torch.float32).reshape(8,4)[None,None,:,:].repeat(16,5,1,1).cuda()
print(x.shape)
y = torch.zeros((16, 5, 8, 4)).cuda()
add_one[(2,)](x, y, 5*8*4, 8*4, 4, 1)


torch.Size([16, 5, 8, 4])


In [28]:
x[12,3,:,:],y[12,3,:,:]

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]], device='cuda:0'),
 tensor([[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]], device='cuda:0'))

In [7]:
torch.all(y == 0)

tensor(True, device='cuda:0')

$ softmax(x_i) = \frac{e^{x_i -m}}{\sum_{i=1}^{N}e^{x_i-m}}  $ 


R=$
\begin{bmatrix}
cos\theta & sin\theta \\
-sin \theta & cos\theta  \\
\end{bmatrix}
$


$$
R^T = 
\begin{bmatrix}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta
\end{bmatrix}
=
\begin{bmatrix}
\cos(-\theta) & \sin(-\theta) \\
-\sin(-\theta) & \cos(-\theta)
\end{bmatrix}
= R(-\theta)
$$